# Data Cleaning and Processing

This notebook processes and cleans the two raw data files from 2015 and 2016.

In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

## Read Data

In [2]:
counted_2015 = pd.read_csv('../data/raw/the-counted-2015.csv')
counted_2016 = pd.read_csv('../data/raw/the-counted-2016.csv')

# Combine data into one dataframe
counted = pd.concat([counted_2015, counted_2016], ignore_index=True)

## Data Cleaning

Below corrects the typos in the `age` and `city` columns.

In [3]:
# Data cleaning
counted['age'] = pd.to_numeric(counted['age'].replace('40s', 40), errors='coerce')
counted['city'] = counted['city'].str.replace('Peltzer', 'Pelzer')
counted['city'] = counted['city'].str.replace('Rudioso', 'Ruidoso')
counted['city'] = counted['city'].str.replace('Constantania', 'Constantia')
counted['city'] = counted['city'].str.replace('Sildell', 'Slidell')
counted['city'] = counted['city'].str.replace('Rossvile', 'Rossville')
counted['city'] = counted['city'].str.replace('Wasau', 'Wausau')

This maps all the states abbreviation to their corresponding full names.

In [4]:
state_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
counted['state'] = counted['state'].map(state_dict)

## Data Engineering

Below performs three data engineerings:
- geocode address to latitude and longitude
- combine age into age groups
- combine month, day, year into one date

### Geocode location

In [ ]:
address = counted[['streetaddress', 'city', 'state']].apply(
    lambda x: ', '.join(x.dropna()), axis = 1
)
longitude, latitude = [], []

# Use geopy to geocode addresses 
# This takes a very long time to run
loc = Nominatim(user_agent="Geopy Library")
for ad in address:
    getLoc = loc.geocode(ad, timeout=None)
    if getLoc is None:
        ad = ', '.join(ad.split(', ')[1:])
        getLoc = loc.geocode(ad, timeout=None)
    longitude.append(getLoc.longitude)
    latitude.append(getLoc.latitude)

counted['latitude'] = latitude
counted['longitude'] = longitude

### Engineer age groups

In [10]:
# Engineer age group
counted.loc[counted['age']<=19, 'age_group'] = 'Under 19'
counted.loc[counted['age'].between(20,39), 'age_group'] = '20-39'
counted.loc[counted['age'].between(40,59), 'age_group'] = '40-59'
counted.loc[counted['age']>=60, 'age_group'] = 'Above 60'
counted.loc[counted['age'].isna(), 'age_group'] = np.nan

### Engineer datetime

In [11]:
# Engineer datetime
counted['date'] = pd.to_datetime(counted[['day', 'month', 'year']].astype(str).agg('-'.join, axis=1), format='%d-%B-%Y')

### Impute Unknwon race/ethnicit and armed to Other

In [17]:
counted.loc[counted['raceethnicity'] == 'Unknown', 'raceethnicity'] = 'Other'
counted.loc[counted['armed'] == 'Unknown', 'armed'] = 'Other'

## Rearrange and Drop Columns for Tidier Outputs

In [18]:
cols = ['age', 'age_group', 'gender', 'raceethnicity', 'month', 'day', 'year', 'date', 'city', 'state', 'latitude', 'longitude', 'classification', 'armed']
counted = counted[cols]
counted.head()

,age,age_group,gender,raceethnicity,month,day,year,date,city,state,latitude,longitude,classification,armed
0,22.0,20-39,Male,Black,January,1,2015,2015-01-01,Savannah,Georgia,32.066828,-81.166272,Death in custody,No
1,47.0,40-59,Male,White,January,2,2015,2015-01-02,Aloha,Oregon,45.487341,-122.892073,Gunshot,Firearm
2,19.0,Under 19,Male,White,January,3,2015,2015-01-03,Kaumakani,Hawaii,21.918289,-159.620750,Struck by vehicle,No
3,23.0,20-39,Male,Hispanic/Latino,January,3,2015,2015-01-03,Wichita,Kansas,37.744441,-97.280718,Gunshot,No
4,53.0,40-59,Male,Asian/Pacific Islander,January,2,2015,2015-01-02,Shelton,Washington,47.245850,-123.120086,Gunshot,Firearm


## Output Clean Data

This writes the clean data onto the `data/proc/clean_data.csv`.

In [19]:
counted.to_csv('../data/proc/clean_data.csv', index=False)  